## 個別ページのデータ収集

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import jaconv

YEAR = 2020 #現状は2020年で決め打ち

In [2]:
def get_inspection_and_patient(s): # PCR検査数と陽性数を返す
    search = re.compile("^(?=.*結果：陰性).*$")
    inspection_element = s.find("p", text=search) # 変数searchで指定する正規表現に一致するpタグを取り出す

    inspections = re.findall(r'[0-9]+', jaconv.z2h(inspection_element.text.replace("，", ""), kana=False, digit=True, ascii=True))  # 対象のpタグから数値を取り出す

    inspection_num = inspections[0]

    patient_num = inspections[2]  if len(inspections) > 2 else 0 #県のサイトでは陽性が出ていないときは陰性の数しか書かれていないため
    
    return inspection_num, patient_num

In [3]:
def get_quarent(s): # 相談件数を返す
    search = re.compile("^(全県相談件数：).*$")
    quarent_element = s.find("p", text=search) # 変数searchで指定する正規表現に一致するpタグを取り出す

    quarents = re.findall(r'[0-9]+', jaconv.z2h(quarent_element.text.replace("，", ""), kana=False, digit=True, ascii=True))  # 対象のpタグから数値を取り出す

    quarent_num = quarents[0]

    return quarent_num

In [4]:
def get_timestamp(s): # その記事内のタイムスタンプを返す
    search = re.compile("^(?=.*までの件数は次のとおりです).*$")
    date_element = s.find("p", text=search)

    date = re.findall(r'[0-9]+', jaconv.z2h(date_element.text.replace("，", ""), kana=False, digit=True, ascii=True)) # 対象のpタグから数値を取り出す

    time_stamp = "{}-{}-{}T08:00:00.000Z".format(YEAR, date[0].zfill(2), date[1].zfill(2)) # 現状は8時決め打ちで、json記載のタイムスタンプに整形している
    
    return time_stamp

## ニュース一覧から新型コロナウイルスの件数ページを取得する

更新記事数が20件を超えるとページネーションされるので注意

In [5]:
# 報道発表ページで「新型コロナウイルス感染症にかかる」で検索した際の結果を利用
res = requests.get("https://webnavi.pref.yamaguchi.lg.jp/press/?q=%E6%96%B0%E5%9E%8B%E3%82%B3%E3%83%AD%E3%83%8A%E3%82%A6%E3%82%A4%E3%83%AB%E3%82%B9%E6%84%9F%E6%9F%93%E7%97%87%E3%81%AB%E3%81%8B%E3%81%8B%E3%82%8B")
res.encoding = res.apparent_encoding 
soup = BeautifulSoup(res.content, "html.parser")

In [6]:
search = re.compile("^(?=.*新型コロナウイルス感染症にかかる相談件数等について).*$")
news_link_element = soup.find_all("a", text=search) # 個別記事のaタグを取り出す

In [8]:
# 各記事のaタグのhref属性からスクレイピングを行う
for elem in news_link_element:
    print(elem.attrs['href'])
    res = requests.get(elem.attrs['href'])
    res.encoding = res.apparent_encoding 
    soup = BeautifulSoup(res.content, "html.parser")
    print("検査数, 陽性数 ： ", get_inspection_and_patient(soup))
    print("相談件数 ： ", get_quarent(soup))
    print("タイムスタンプ ： ", get_timestamp(soup))

https://www.pref.yamaguchi.lg.jp/press/202003/045875.html
検査数, 陽性数 ：  ('149', '3')
相談件数 ：  6268
タイムスタンプ ：  2020-03-17T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045860.html
検査数, 陽性数 ：  ('136', '3')
相談件数 ：  5846
タイムスタンプ ：  2020-03-15T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045850.html
検査数, 陽性数 ：  ('122', '3')
相談件数 ：  5510
タイムスタンプ ：  2020-03-12T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045849.html
検査数, 陽性数 ：  ('113', '3')
相談件数 ：  5298
タイムスタンプ ：  2020-03-11T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045834.html
検査数, 陽性数 ：  ('104', '3')
相談件数 ：  5076
タイムスタンプ ：  2020-03-10T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045827.html
検査数, 陽性数 ：  ('88', '3')
相談件数 ：  4375
タイムスタンプ ：  2020-03-08T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045819.html
検査数, 陽性数 ：  ('77', '3')
相談件数 ：  3806
タイムスタンプ ：  2020-03-05T08:00:00.000Z
https://www.pref.yamaguchi.lg.jp/press/202003/045808.html
検査数, 陽性数 ：  ('51', '

# ToDo
- jsonに格納する
- 都度すべてをスクレイピングすると効率悪いので差分だけスクレイピングするようにしたい
- hostingしてからどうするかは別途相談
- 20件以上になったときの対応